# Setup
import all the libraries and functions we will need ..

In [1]:
import cv2                              # for image processing
import face_recognition                 # for face detection, encoding and recognition

import os                               # for file handling (image directory)
import glob                             # to take all images from directory
import json                             # to store data in JSON format

from numpy import array, argmin         # to get min distance between known face and unknown face


# Store known Data
some lists to store information about the people we want to know from the video, wheteher they are them or not ..

In [2]:
known_face_encodings    = []            # to store known face encodings
known_face_names        = []            # to store known face names          ***
frame_resize            = 0.25          # to resize frame

# Load Dataset
all images of known people

In [3]:
images_path = glob.glob(os.path.join("images 2/*.*"))  # get all images from directory

# Get information
get name and feature vector for each image to compare them and and find out its name

In [4]:
for img_path in images_path:                                # for each image

    img      = cv2.imread(img_path)                         # read image
    
    small_img = cv2.resize(img, (0, 0), fx = frame_resize, fy = frame_resize)

    rgb_img  = cv2.cvtColor(small_img, cv2.COLOR_BGR2RGB)

    basename = os.path.basename(img_path)                   # get full image name       ***

    filename, ext = os.path.splitext(basename)              # split to name and extension   ***
    
    img_encoding  = face_recognition.face_encodings(rgb_img)[0] # get encoding

    known_face_encodings.append(img_encoding)               # append encoding
    
    known_face_names.append(filename)                       # append name     ***

# If the same person 
write **True** value on Verified 

In [5]:
def read_shared_variable():
    with open('F:\\VS\\ML\\Sub Projects For GP\\Face Recognition\\face_recognition lib\\data.json', 'r') as f:
        data = json.load(f)
        verified = data['Verified']
    return verified

def write_shared_variable():
    with open('F:\\VS\\ML\\Sub Projects For GP\\Face Recognition\\face_recognition lib\\data.json', 'r') as f:
        data = json.load(f)

    data['Verified'] = True

    with open('F:\\VS\\ML\\Sub Projects For GP\\Face Recognition\\face_recognition lib\\data.json', 'w') as f:
        json.dump(data, f)


# Detect faces function
get all faces and their names from in the frame and compare them using distance 

the minimum distance between him and the known people **will most likely be correct**


In [6]:
def detect_faces(frame):
    
    small_frame = cv2.resize(frame, (0, 0), fx = frame_resize, fy = frame_resize)

    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_small_frame)                   # get face locations

    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)   # get face encodings

    face_names = []                                                                     # to store face names for each face in the frame
    
    for face_encoding in face_encodings:

        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)   # compare face encodings with known face encodings, return True if they match

        name = "Unknown"                                                                # if unknown face, name = Unknown

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)    # get min distance between known face and unknown face

        best_match_index = argmin(face_distances)                                       # get index of min distance

        if matches[best_match_index]:                                                   # if unknown face matches with known face
            write_shared_variable()                                                     # write True
            name = known_face_names[best_match_index]                                   # assign name by known face name
            
        face_names.append(name)

    face_locations = array(face_locations)                                              # convert to array to do operations quickly (using list will not work)

    face_locations = face_locations / frame_resize                                      # resize face locations

    return face_locations.astype(int), face_names                                       # return all face locations and there names in the frame



# Test 

In [7]:
cap = cv2.VideoCapture(0)                                                               # camera setup   

while True:                                                             
    ret, frame = cap.read()                                                             # read frame by frame

    face_locations, face_names = detect_faces(frame)                                    # using detect faces function to get face locations and names in the frame

    for face_loc, name in zip(face_locations, face_names):                              # for each face in the frame

        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]             # get face location to draw rectangle around face (top, right, bottom, left)

        cv2.putText(frame, name,(x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 200), 2)  # put name above face
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 2)                        # draw rectangle

    cv2.imshow("Frame", frame)                                                          # show frame after processing

    verified = read_shared_variable()
    if verified:
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):                                               # press q to quit
        break

cap.release()                                                                           # release camera
cv2.destroyAllWindows()                                                                 # close all windows (imshow windows)
